# Настройка среды

In [2]:
import os
import sys
sys.path.append(os.path.abspath("../.."))

In [3]:
%pip install --upgrade pip
%pip install optuna
%pip install belashovplot
%pip install belashovplot
%pip install timm

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.6 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip

# Импорты и настройка модулей

In [4]:
from belashovplot import TiledPlot
from utilities import *
from utilities.filters import Gaussian, Window
from utilities.training import train, confusion
from elements.modulators import Lens, PhaseModulator, AmplitudeModulator
from elements.propagators import FurrierPropagation, ConvolutionalPropagation
from elements.composition import CompositeModel, HybridModel
from elements.wrappers import CudaMemoryChunker, Incoherent
from elements.detectors import ClassificationDetectors, MatrixDetectors
from parameters import FigureWidthHeight, FontLibrary
from tqdm import tqdm
from math import sin, sqrt
from pickle import dump
import torch
import numpy
import timm
import optuna
import pandas

In [5]:
FontLibrary.Fonts.ColumnDescriptionTop.FontSize = 9
FontLibrary.Fonts.ColumnDescriptionBottom.FontSize = 9
FontLibrary.Fonts.RowDescriptionLeft.FontSize = 9
FontLibrary.Fonts.RowDescriptionRight.FontSize = 9
FigureWidthHeight = (16, 16)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Используемый девайс:', torch.cuda.get_device_name(device) if torch.cuda.is_available() else 'ЦП')

Используемый девайс: Tesla T4


# Инициализация вспомогательных методов

## Комбинированная лосс функция

In [7]:
def combined_loss(cross_entropy_to_mse_proportion:float=1.0):
    def loss_function(outputs, targets):
        CELoss = torch.nn.functional.cross_entropy(outputs, targets)
        MSELoss = torch.nn.functional.mse_loss(torch.nn.functional.softmax(outputs, dim=1), torch.nn.functional.one_hot(targets, num_classes=10).float())
        loss = cross_entropy_to_mse_proportion*CELoss + (1.0-cross_entropy_to_mse_proportion)*MSELoss
        return loss
    return loss_function

## Электронная моель ResNet

In [8]:
def resnet18(classes:int=10):
    return timm.create_model('resnet18', pretrained=False,  in_chans=1, num_classes=classes).to(device)

## Инициализация системы по полному набору параметров

In [9]:
def initialize(
    wavelength:float, N:int, length:float, pixels:int, masks_amount:int, distance:float,
    spatial_coherence:float, time_coherence:float, time_relaxation:float, mean_samples:int,
    detector_size:float, detectors_amount:int,
    loss_function_proportion:float,  batch_size:int, optimizer_type:str
):
    print("Инициализированна модель со следующими параметрами:")
    print(f"\tДлинна волны:                             {engineering(wavelength, 'м')}")
    print(f"\tКоличество вычислительных пикселей:       {N}")
    print(f"\tКоличество пикселей маски:                {pixels}")
    print(f"\tРазмер оптических элементов:              {engineering(length, 'м')}")
    print(f"\tРазмер пикселя маски:                     {engineering(length/pixels, 'м')}")
    print(f"\tРасстояние между слоями:                  {engineering(distance, 'м')}")
    
    print(f"\tВременная когерентность:                  {engineering(time_coherence, 'с')}")
    print(f"\tВремя релаксации:                         {engineering(time_relaxation, 'c')}")
    print(f"\tПространственная когерентность:           {engineering(spatial_coherence, 'м')}")
    print(f"\tКоличество усреднений:                    {mean_samples}")

    print(f"\tРазмер детекторов:                        {engineering(detector_size, 'м')}")
    print(f"\tКоличество детекторов:                    {detectors_amount} на {detectors_amount}")

    print(f"\tПропорция CE к MSE лосс:                  {loss_function_proportion}")
    print(f"\tРазмер батча:                             {batch_size}")
    print(f"\tТип оптимизатора:                         {optimizer_type}")
    print()
    
    incoherent = Incoherent(spatial_coherence, time_coherence, time_relaxation, mean_samples, N, length).to(device)

    spectral_filter = Window(centers=wavelength, sizes=300.0E-9)
    detectors_filter = Gaussian((detector_size, detector_size), (0,0))
    detectors = MatrixDetectors(N, length, wavelength, detectors_amount, detectors_filter, spectral_filter).to(device)

    electronic = resnet18()

    propagation = FurrierPropagation(N, length, wavelength, 1.0, 0.0, distance, 0.4)
    phase_modulators = [PhaseModulator(N, length, pixels) for i in range(masks_amount)]
    amplitude_modulators = [AmplitudeModulator(N, length, pixels) for i in range(masks_amount)]
    elements = [phase_modulators[0], amplitude_modulators[0]]
    for phase_modulator, amplitude_modulator in zip(phase_modulators[1:], amplitude_modulators[1:]):
        elements.append(propagation)
        elements.append(phase_modulator)
        elements.append(amplitude_modulator)
    elements.append(propagation)
    chunker = CudaMemoryChunker()
    optical = CompositeModel(*elements)
    optical.wrap(chunker)
    optical.wrap(incoherent)
    optical.to(device)
    model = HybridModel(optical, detectors, electronic).to(device)

    dataset = Dataset('CIFAR10', batch_size, N, N, torch.complex64)
    dataset.train
    dataset.test
    
    loss_function = combined_loss(loss_function_proportion)
    optimizer_types_list = {'Adam':torch.optim.Adam, 'SGD':torch.optim.SGD, 'RMSprop':torch.optim.RMSprop, 'Adagrad':torch.optim.Adagrad}
    optimizer_type = optimizer_types_list[optimizer_type]

    return model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic)

# Установка не изменяемых параметров системы

In [10]:
# Предпочтительные параметры
size = 50.0E-6
near_N = 300
near_length = 5.0E-3
wavelength = 500.0E-9
detectors_amount = 24

# Параметры когерентности
spatial_coherence = 50.0E-6
time_coherence = 10.0E-9
time_relaxation = 1.0E-6
mean_samples = 7

In [11]:
# Вычисляемые параметры
pixels = upper_integer(near_length/size)
length = pixels * size
cppp = upper_integer(near_N * size / length)
N = int(length / size) * cppp
detector_size = length / 60

# Отображение базы данных тренировок

In [12]:
study = optuna.create_study(study_name="ID2NN", storage="sqlite:///ID2NN.db", direction='maximize', load_if_exists=True)
dataframe:pandas.DataFrame = study.trials_dataframe()
print(f"Кол-во исследований: {len(study.trials)}")

[I 2024-05-04 18:12:33,438] Using an existing study with name 'ID2NN' instead of creating a new one.


Кол-во исследований: 265


In [13]:
dataframe.nlargest(5, 'value')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
108,108,30.02,2024-05-02 22:18:41.949319,2024-05-02 22:30:25.700272,0 days 00:11:43.750953,36,0.931286,0.080028,0.005589,6,RMSprop,COMPLETE
109,109,29.42,2024-05-02 22:30:25.719252,2024-05-02 22:42:09.999907,0 days 00:11:44.280655,36,0.992874,0.080766,0.005603,6,RMSprop,COMPLETE
261,261,29.29,2024-05-04 19:52:31.046571,2024-05-04 20:04:16.509982,0 days 00:11:45.463411,37,0.785131,0.103529,0.003100,5,Adam,COMPLETE
264,264,29.04,2024-05-04 20:26:50.107605,2024-05-04 20:38:10.500097,0 days 00:11:20.392492,38,0.793974,0.103155,0.004389,5,Adam,COMPLETE
242,242,29.03,2024-05-04 16:33:44.271063,2024-05-04 16:43:33.993515,0 days 00:09:49.722452,36,0.913746,0.096982,0.002657,5,Adam,COMPLETE


In [14]:
dataframe[dataframe['value'] >= 29].nlargest(5, 'params_distance')

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
261,261,29.29,2024-05-04 19:52:31.046571,2024-05-04 20:04:16.509982,0 days 00:11:45.463411,37,0.785131,0.103529,0.003100,5,Adam,COMPLETE
264,264,29.04,2024-05-04 20:26:50.107605,2024-05-04 20:38:10.500097,0 days 00:11:20.392492,38,0.793974,0.103155,0.004389,5,Adam,COMPLETE
242,242,29.03,2024-05-04 16:33:44.271063,2024-05-04 16:43:33.993515,0 days 00:09:49.722452,36,0.913746,0.096982,0.002657,5,Adam,COMPLETE
109,109,29.42,2024-05-02 22:30:25.719252,2024-05-02 22:42:09.999907,0 days 00:11:44.280655,36,0.992874,0.080766,0.005603,6,RMSprop,COMPLETE
108,108,30.02,2024-05-02 22:18:41.949319,2024-05-02 22:30:25.700272,0 days 00:11:43.750953,36,0.931286,0.080028,0.005589,6,RMSprop,COMPLETE


# Модель №1

In [15]:
# Выберем самый лучший вариант 108
dataframe[dataframe['number'] == 108]

,number,value,datetime_start,datetime_complete,duration,params_batch size,params_cross entorpy proportion,params_distance,params_learning rate,params_masks amount,params_optimizer type,state
108,108,30.02,2024-05-02 22:18:41.949319,2024-05-02 22:30:25.700272,0 days 00:11:43.750953,36,0.931286,0.080028,0.005589,6,RMSprop,COMPLETE


## Утсановка оптимизированных параметров системы

In [16]:
masks_amount = 6
batch_size = 36
distance = 0.08
learning_rate = 0.005589
loss_function_proportion = 0.931286
optimizer_type = 'RMSprop'

In [17]:
model, dataset, loss_function, optimizer_type, (incoherent, chunker, optical, detectors, electronic) = initialize(
    wavelength, N, length, pixels, masks_amount, distance,
    spatial_coherence, time_coherence, time_relaxation, mean_samples,
    detector_size, detectors_amount,
    loss_function_proportion,  batch_size, optimizer_type
)

Инициализированна модель со следующими параметрами:
	Длинна волны:                             500.0 нм
	Количество вычислительных пикселей:       300
	Количество пикселей маски:                100
	Размер оптических элементов:              5.0 мм
	Размер пикселя маски:                     50.0 мкм
	Расстояние между слоями:                  80.0 мм
	Временная когерентность:                  10.0 нс
	Время релаксации:                         1.0 мкc
	Пространственная когерентность:           50.0 мкм
	Количество усреднений:                    7
	Размер детекторов:                        83.333 мкм
	Количество детекторов:                    24 на 24
	Пропорция CE к MSE лосс:                  0.931286
	Размер батча:                             36
	Тип оптимизатора:                         RMSprop

Files already downloaded and verified
Files already downloaded and verified


## Тренировка

In [18]:
loss_histories = []
cofusion_matrixes = []

In [ ]:
optimizer = optimizer_type(model.parameters(), lr=learning_rate)
cofusion_matrixes.append(confusion(model, dataset))
print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")
for i in range(10):
    loss_histories.append(train(model, dataset, optimizer, loss_function))
    cofusion_matrixes.append(confusion(model, dataset))
    print(f"Точность: {100*numpy.sum(numpy.diagonal(cofusion_matrixes[-1], 0))/numpy.sum(cofusion_matrixes[-1])}%")

100%|██████████| 278/278 [01:41<00:00,  2.75it/s]


Точность: 10.0%


RLoss: 2.0045414852488435, RPI1000: -0.22099296374655575: 100%|██████████| 1389/1389 [17:38<00:00,  1.31it/s]
100%|██████████| 278/278 [01:39<00:00,  2.80it/s]


Точность: 21.65%


RLoss: 1.952499848216772, RPI1000: -0.16558760326646996:   8%|▊         | 105/1389 [01:19<16:19,  1.31it/s] 